In [1]:
import re
import json
import pickle
import os
import sys
import requests
import logging
import torch
from tqdm.auto import tqdm
from transformer_lens import HookedTransformer
from tqdm.auto import tqdm
import plotly.io as pio
import numpy as np
import random
import torch.nn as nn
import torch.nn.functional as F
import wandb
import plotly.express as px
import pandas as pd
import torch.nn.init as init
from pathlib import Path
from jaxtyping import Int, Float
from torch import Tensor
import einops
from collections import Counter
from datasets import load_dataset
import pandas as pd
from ipywidgets import interact, IntSlider
from process_tiny_stories_data import load_tinystories_validation_prompts, load_tinystories_tokens

pio.renderers.default = "notebook_connected"
device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"
torch.autograd.set_grad_enabled(False)
torch.set_grad_enabled(False)

logging.basicConfig(format='(%(levelname)s) %(asctime)s: %(message)s', level=logging.INFO, datefmt='%I:%M:%S')
sys.path.append('../')  # Add the parent directory to the system path

import utils.haystack_utils as haystack_utils
from sparse_coding.train_autoencoder import AutoEncoder
from utils.autoencoder_utils import custom_forward, AutoEncoderConfig, evaluate_autoencoder_reconstruction, get_encoder_feature_frequencies, load_encoder
import utils.haystack_utils as haystack_utils

%reload_ext autoreload
%autoreload 2

In [ ]:
# Load interesting encoder, find token features, generate histogram of activations for that token. Park features.

In [6]:
# Run overview
model_name = "tiny-stories-2L-33M"
layer_name = "L0"
print_model_name = f"{model_name}-{layer_name}"

model = HookedTransformer.from_pretrained(
        model_name,
        center_unembed=True,
        center_writing_weights=True,
        fold_ln=True,
        device=device,
    )

prompts = load_tinystories_validation_prompts()

Using pad_token, but it is not set yet.


Loaded pretrained model tiny-stories-2L-33M into HookedTransformer


In [17]:
save_names = [f.split(".")[0] for f in os.listdir(model_name) if f.endswith('.pt')]
l0_encoder, l0_cfg = load_encoder('18_morning_sun', model_name, model)

In [26]:
@torch.no_grad()
def get_acts(prompt: str, model: HookedTransformer, encoder: AutoEncoder, cfg: AutoEncoderConfig):
    _, cache = model.run_with_cache(prompt, names_filter=cfg.encoder_hook_point)
    acts = cache[cfg.encoder_hook_point].squeeze(0)
    _, _, mid_acts, _, _ = encoder(acts)
    return mid_acts


def get_max_activations(prompts: list[str], model: HookedTransformer, encoder: AutoEncoder, cfg: AutoEncoderConfig):
    activations = []
    indices = []
    for prompt in tqdm(prompts):
        # get encoder activations at final position
        acts = get_acts(prompt, model, encoder, cfg)[0, :-1]
        activations.append(acts)

    max_activation_per_prompt = torch.stack(activations)  # n_prompt x d_enc
    max_activation_token_index = torch.stack(indices)

    total_activations = max_activation_per_prompt.sum(0)
    print(f"Active directions on validation data: {total_activations.nonzero().shape[0]} out of {total_activations.shape[0]}")
    return max_activation_per_prompt, max_activation_token_index

def print_top_examples(prompts: list[str], activations: Float[Tensor, "n_prompts d_enc"], direction: int, cfg: AutoEncoderConfig, encoder: AutoEncoder, n=5):
    top_idxs = activations[:, direction].argsort(descending=True)[:n].cpu().tolist()
    for prompt_index in top_idxs:
        prompt = prompts[prompt_index]
        prompt_tokens = model.to_str_tokens(model.to_tokens(prompt))
        acts = get_acts(prompt, model, encoder, cfg)
        direction_act = acts[:, direction].cpu().tolist()
        max_direction_act = max(direction_act)
        if max_direction_act > 0:
            haystack_utils.clean_print_strings_as_html(prompt_tokens, direction_act, max_value=max_direction_act)

max_activations, max_activation_token_indices = get_max_activations(prompts, model, l0_encoder, l0_cfg)

  0%|          | 0/21990 [00:00<?, ?it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 94.00 MiB. GPU 0 has a total capacty of 47.54 GiB of which 89.75 MiB is free. Process 5405 has 16.40 GiB memory in use. Process 32528 has 31.06 GiB memory in use. Of the allocated memory 27.63 GiB is allocated by PyTorch, and 2.34 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
prompt = "One day, a little girl named Lily went for a walk in the park"
acts = get_acts(prompt, model, l0_encoder, l0_cfg)[-1] # d_enc
print(f"Active directions on last token: {acts.nonzero().shape[0]} out of {acts.shape[0]}")
active_directions = acts.nonzero().squeeze(1)
highly_active_directions = torch.argwhere((acts > 0.5)).squeeze(1)
for active_direction in highly_active_directions:
    print(f"Direction {active_direction}")
    print_top_examples(prompts, max_activations, active_direction, l0_cfg, l0_encoder, 4)

In [ ]:
token = model.to_single_token(" park")
token_prompts = []
for prompt in prompts[:1000]:
    tokens = model.to_tokens(prompt)
    if token in tokens:
        token_prompts.append(prompt)
print(len(token_prompts))

In [ ]:
def plot_direction_frequency(data: list[str], direction: int, cfg: AutoEncoderConfig, encoder: AutoEncoder):
    activations = []
    for prompt in tqdm(data):
        tokens = model.to_tokens(prompt)
        _, cache = model.run_with_cache(
            tokens, names_filter=f"blocks.{cfg.layer}.{cfg.act_name}"
            )
        acts = cache[f"blocks.{cfg.layer}.{cfg.act_name}"].squeeze(0)
        _, _, mid_acts, _, _ = encoder(acts)
        activations.append(mid_acts[:, direction])
    activations = torch.cat(activations)
    print(activations.shape)

    fig = px.histogram(activations.tolist(), 
                       title=f"{print_model_name} L1={cfg.l1_coeff}: Activations for direction {direction}", 
                       histnorm="probability")
    fig.update_layout(
        xaxis_title="Activation",
        yaxis_title="Probability",
        width = 600,
        showlegend=False
    )
    fig.update_yaxes(type='log')
    fig.show()
    
for direction in active_directions[:10]:
    plot_direction_frequency(prompts[:50], direction, l0_cfg, l0_encoder)